In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 1. Import the dataset and clean column names
data = pd.read_csv("RealEstate_Prices.csv")
data.head()

,Property ID,Property Type,Location,Bedrooms,Bathrooms,Square Feet,Year Built,Sale Price,Neighborhood,Amenities Distance Miles
0,1,House,Downtown,2,3,1148,2009.0,807870.0,Neighborhood A,4.745561
1,2,Apartment,Suburb,5,1,2663,2019.0,776389.0,Neighborhood C,1.453618
2,3,House,Downtown,4,3,2957,NaN,NaN,Neighborhood A,2.480775
3,4,House,Rural,4,3,2721,2001.0,186148.0,Neighborhood C,0.141468
4,5,Apartment,Downtown,2,1,2780,2009.0,405378.0,Neighborhood B,0.305964


In [11]:
# Clean column names by removing spaces, special characters, and renaming for clarity
data.columns = data.columns.str.strip().str.replace(' ', '_').str.replace(r'[^\w\s]', '')


In [12]:
# Display the first few rows to verify column cleaning
print("Cleaned Column Names:")
print(data.head())

Cleaned Column Names:
   Property_ID Property_Type  Location  Bedrooms  Bathrooms  Square_Feet  \
0            1         House  Downtown         2          3         1148   
1            2     Apartment    Suburb         5          1         2663   
2            3         House  Downtown         4          3         2957   
3            4         House     Rural         4          3         2721   
4            5     Apartment  Downtown         2          1         2780   

   Year_Built  Sale_Price    Neighborhood  Amenities_Distance_Miles  
0      2009.0    807870.0  Neighborhood A                  4.745561  
1      2019.0    776389.0  Neighborhood C                  1.453618  
2         NaN         NaN  Neighborhood A                  2.480775  
3      2001.0    186148.0  Neighborhood C                  0.141468  
4      2009.0    405378.0  Neighborhood B                  0.305964  


In [13]:
# 2. Handle missing values
# Decide on a strategy to handle missing values. Example: filling numerical with mean and categorical with mode.
for column in data.columns:
    if data[column].dtype == 'object':  # Check if column is categorical
        data[column].fillna(data[column].mode()[0], inplace=True)
    else:  # Numeric column
        data[column].fillna(data[column].mean(), inplace=True)

# Display summary of missing values to ensure they are handled
print("\nMissing Values after Imputation:")
print(data.isnull().sum())


Missing Values after Imputation:
Property_ID                 0
Property_Type               0
Location                    0
Bedrooms                    0
Bathrooms                   0
Square_Feet                 0
Year_Built                  0
Sale_Price                  0
Neighborhood                0
Amenities_Distance_Miles    0
dtype: int64


In [15]:
# 4. Filter and subset data based on criteria
# Example: Filter properties sold after 2015 and of type 'Single Family'
filtered_data = data[(data['Year_Built'] > 2015) & (data['Property_Type'] == 'Single Family')]

In [16]:
# 5. Handle categorical variables by encoding
# Example of one-hot encoding categorical variables
encoded_data = pd.get_dummies(filtered_data, columns=['Property_Type', 'c'], drop_first=True)

In [27]:
# Print the column names to identify the correct one
print("Current Columns in encoded_data:")
print(encoded_data.columns)

Current Columns in encoded_data:
Index(['Property_ID', 'Location', 'Bedrooms', 'Bathrooms', 'Square_Feet',
       'Year_Built', 'Sale_Price', 'Amenities_Distance_Miles'],
      dtype='object')


In [30]:
# Assuming the relevant column for aggregation is 'Neighborhood' and the target is 'Sale_Price'
# Replace 'Neighborhood' and 'Sale_Price' with the actual column names if they are different
avg_price_by_neighborhood = data.groupby('Neighborhood').agg({'Sale_Price': 'mean'}).reset_index()
avg_price_by_neighborhood.rename(columns={'Sale_Price': 'Average_Sale_Price'}, inplace=True)

# Display the result
print("\nAverage Sale Price by Neighborhood:")
print(avg_price_by_neighborhood)


Average Sale Price by Neighborhood:
     Neighborhood  Average_Sale_Price
0  Neighborhood A       533672.207207
1  Neighborhood B       542188.843137
2  Neighborhood C       642901.030651


In [19]:
# 7. Identify and handle outliers
# Using IQR to identify outliers in 'Sale_Price'
Q1 = encoded_data['Sale_Price'].quantile(0.25)
Q3 = encoded_data['Sale_Price'].quantile(0.75)
IQR = Q3 - Q1

# Define limits for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
outlier_filtered_data = encoded_data[(encoded_data['Sale_Price'] >= lower_bound) & (encoded_data['Sale_Price'] <= upper_bound)]

# Display number of rows before and after outlier removal
print("\nNumber of rows before outlier removal:", len(encoded_data))
print("Number of rows after outlier removal:", len(outlier_filtered_data))



Number of rows before outlier removal: 0
Number of rows after outlier removal: 0


In [31]:
cleaned_data = outlier_filtered_data
cleaned_data.to_csv("Cleaned_RealEstate_Data.csv", index=False)
print("\nData wrangling completed! Cleaned dataset saved as 'Cleaned_RealEstate_Data.csv'.")


Data wrangling completed! Cleaned dataset saved as 'Cleaned_RealEstate_Data.csv'.
